In [1]:
from code_evaluate_tool import *
from json_tool import *
from filter_tool import *
from llm_config import *

c:\python_dir\Test_Crawler2\llm_config.py:52: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_core = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


### LLM이 동작하는지 간단히 테스트

In [ ]:
response = llm('hello. who are you?')
response

### 코드에 대한 점수를 매기기 위해, 필터링 된 jsonl 파일을 읽어 데이터 프레임 형태로 변환한다.

In [4]:
df = read_jsonl_file("code_filtered/Code_filtered_example.jsonl")
df

,file_name,line_count,code
0,AboutDialog_function_1.cpp,27,"CALLBACK DialogProc(HWND hWnd, UINT uMsg, WPAR..."
1,AboutDialog_function_2.cpp,4,"void About()\n{\n\tDialogBox(g_hInstance, MAKE..."
2,ActiveStatus_function_1.cpp,7,eActiveStatus GetStatusFromString(const std::s...
3,AdditionalFile_function_1.cpp,1,"StringView externallyDefinedFunc() { return ""Y..."
4,address_v4_function_1.cpp,4,address_v4 make_address_v4(const address_v4::b...
...,...,...,...
9752,x11grab_function_4.cpp,28,int\n init_shm() {\n static void *hand...
9753,x11grab_function_5.cpp,31,int\n init() {\n static void *handle {...
9754,x11grab_function_6.cpp,50,"void\n blend_cursor(Display *display, img_t &..."
9755,x11grab_function_7.cpp,66,"int\n init(const std::string &display_name,..."


In [5]:
start = 0
end = df.index.size
print(start, end)

0 9757


### 코드에 대한 점수를 매긴다. 점수가 매겨진 데이터 프레임은 df2에 저장된다.

In [6]:
df2 = assign_mark_with_llm(df, start, end, llm)
df2

  0%|          | 1/9757 [00:00<1:24:36,  1.92it/s]

Saved: df_mark_0.json


100%|██████████| 9757/9757 [1:48:09<00:00,  1.50it/s]  


,file_name,line_count,code,mark
0,AboutDialog_function_1.cpp,27,"CALLBACK DialogProc(HWND hWnd, UINT uMsg, WPAR...",7.5
1,AboutDialog_function_2.cpp,4,"void About()\n{\n\tDialogBox(g_hInstance, MAKE...",7
2,ActiveStatus_function_1.cpp,7,eActiveStatus GetStatusFromString(const std::s...,7
3,AdditionalFile_function_1.cpp,1,"StringView externallyDefinedFunc() { return ""Y...",7
4,address_v4_function_1.cpp,4,address_v4 make_address_v4(const address_v4::b...,8
...,...,...,...,...
9752,x11grab_function_4.cpp,28,int\n init_shm() {\n static void *hand...,5
9753,x11grab_function_5.cpp,31,int\n init() {\n static void *handle {...,6
9754,x11grab_function_6.cpp,50,"void\n blend_cursor(Display *display, img_t &...",6
9755,x11grab_function_7.cpp,66,"int\n init(const std::string &display_name,...",7


In [7]:
convert_df_to_jsonl(df2, "code_filtered/Codes_marked_example_pre.jsonl")

JSONL 파일이 생성되었습니다!


### 점수(mark)의 데이터 타입은 char 형이므로 float로 바꾸기 위해서 pd.to_numeric(..., errors='coerce')를 이용한다. 바뀐 mark 부분은 float 데이터 타입을 갖는다.

In [8]:
df2['mark'] = pd.to_numeric(df2['mark'], errors='coerce')
df2

,file_name,line_count,code,mark
0,AboutDialog_function_1.cpp,27,"CALLBACK DialogProc(HWND hWnd, UINT uMsg, WPAR...",7.5
1,AboutDialog_function_2.cpp,4,"void About()\n{\n\tDialogBox(g_hInstance, MAKE...",7.0
2,ActiveStatus_function_1.cpp,7,eActiveStatus GetStatusFromString(const std::s...,7.0
3,AdditionalFile_function_1.cpp,1,"StringView externallyDefinedFunc() { return ""Y...",7.0
4,address_v4_function_1.cpp,4,address_v4 make_address_v4(const address_v4::b...,8.0
...,...,...,...,...
9752,x11grab_function_4.cpp,28,int\n init_shm() {\n static void *hand...,5.0
9753,x11grab_function_5.cpp,31,int\n init() {\n static void *handle {...,6.0
9754,x11grab_function_6.cpp,50,"void\n blend_cursor(Display *display, img_t &...",6.0
9755,x11grab_function_7.cpp,66,"int\n init(const std::string &display_name,...",7.0


### 데이터 타입 변환 과정에서 제대로 변환되지 않는 경우가 있는데, 이 경우는 LLM이 점수를 매길 때 output이 숫자 외 다른 문자도 출력한 경우이다. 이 경우에는 NaN 이라고 표시되어 있어, 이러한 것들은 제거해야 한다.

In [9]:
# df2에서 mark가 NaN 이라고 되어 있는 것은 버려야 함
df2 = df2[df2['mark'].notna()].reset_index(drop=True)
df2

,file_name,line_count,code,mark
0,AboutDialog_function_1.cpp,27,"CALLBACK DialogProc(HWND hWnd, UINT uMsg, WPAR...",7.5
1,AboutDialog_function_2.cpp,4,"void About()\n{\n\tDialogBox(g_hInstance, MAKE...",7.0
2,ActiveStatus_function_1.cpp,7,eActiveStatus GetStatusFromString(const std::s...,7.0
3,AdditionalFile_function_1.cpp,1,"StringView externallyDefinedFunc() { return ""Y...",7.0
4,address_v4_function_1.cpp,4,address_v4 make_address_v4(const address_v4::b...,8.0
...,...,...,...,...
9752,x11grab_function_4.cpp,28,int\n init_shm() {\n static void *hand...,5.0
9753,x11grab_function_5.cpp,31,int\n init() {\n static void *handle {...,6.0
9754,x11grab_function_6.cpp,50,"void\n blend_cursor(Display *display, img_t &...",6.0
9755,x11grab_function_7.cpp,66,"int\n init(const std::string &display_name,...",7.0


### 점수화된 최종 jsonl 파일을 저장한다

In [10]:
convert_df_to_jsonl(df2, "code_filtered/Codes_marked_example.jsonl")

JSONL 파일이 생성되었습니다!


### 점수가 8.0 이상인 데이터만 추출하여 (필터링하여) 따로 저장한다. 이 과정은 데이터의 품질을 높여준다.

In [11]:
###  mark 값이 8.0 이상인 것만 필터링
df3 = filter_by_gte_mark(df2, 8.0)
df3

,file_name,line_count,code,mark
0,address_v4_function_1.cpp,4,address_v4 make_address_v4(const address_v4::b...,8.0
1,address_v6_function_2.cpp,9,"void combine_4_bytes(std::size_t& seed, const ...",8.0
2,AlgorithmFind_function_1.cpp,12,"ForwardIterator findIf(ForwardIterator first, ...",8.0
3,Allocator_function_3.cpp,3,"void deallocate(TYPE* p, std::size_t n) {\n ...",8.0
4,AnyMapHolder_function_1.cpp,3,AnyValue create_AnyValue() {\n return AnyValu...,8.0
...,...,...,...,...
1480,wrapped_swapchain_function_5.cpp,4,"STDMETHODCALLTYPE GetPrivateData(REFGUID Name,...",8.0
1481,wrapped_swapchain_function_6.cpp,4,"STDMETHODCALLTYPE GetParent(REFIID riid, void*...",8.0
1482,wrapped_swapchain_function_7.cpp,4,STDMETHODCALLTYPE GetDesc(DXGI_SWAP_CHAIN_DESC...,8.0
1483,wrapped_swapchain_function_8.cpp,4,STDMETHODCALLTYPE ResizeTarget(const DXGI_MODE...,8.0


### 8.0 이상의 점수를 받은 코드들만 jsonl 형태로 저장한다.

In [12]:
convert_df_to_jsonl(df3, "code_filtered/Codes_marked_example_filtered_gte8.0.jsonl")

JSONL 파일이 생성되었습니다!
